In [2]:

import pandas as pd
import numpy as np
import xarray as xr

import glob
import os


from dateutil.relativedelta import *
from datetime import datetime, timedelta, date 

import fastparquet

import gc


import warnings 

warnings.filterwarnings("ignore")

In [4]:
root = '/mnt/c/MBA/data2/'

dataframe = pd.DataFrame()

area = '-10.25/-40.75/-10.75/-40.25'

os.system(f'rm {root}*.idx')

data_1 = np.datetime64('1940-01-01')

while data_1 <  np.datetime64('2023-04-12'):

    print(np.datetime_as_string(data_1, unit='D'))

    os.system(f'python {root}GetERA5-sl.py {data_1.astype(object).year} {data_1.astype(object).month} {data_1.astype(object).day} {area} {root}files_{data_1}.grib ')     

    list_files = glob.glob(f'{root}/files*') 

    clima = xr.open_dataset(list_files[0], engine='cfgrib').drop(['number', 'surface', 'step', 'valid_time']) 

    clima = (clima
            .rename_dims({"time": "date_ref"})
            .rename_vars({"time": "date_ref"})
        )

    clima['WS100'] = (clima['u100']*clima['u100'] + clima['v100'] * clima['v100']) ** 0.5

    clima = clima.sel(latitude=-10.46126756, longitude=-40.5174951, method = "nearest")


    clima = clima.drop_vars(['u100', 'v100', 'latitude', 'longitude'])

    clima = clima.set_index(date_ref="date_ref").sortby("date_ref")

    clima = clima.resample(date_ref='1D').mean()

    clima['date_ref'] = clima.indexes['date_ref'].normalize()

    df = clima.to_dataframe().reset_index()

    dataframe = pd.concat([dataframe, df], axis=0)

    del clima

    gc.collect()

    data=np.datetime_as_string(data_1, unit='D')

    dataframe.to_parquet(f'{root}{data}.parquet', engine='fastparquet')

    data_1 = data_1 + np.timedelta64(1, 'D') 

    os.system(f'rm {root}files*')


rm: cannot remove '/mnt/c/MBA/data2/*.idx': No such file or directory


1940-01-01


2023-04-16 15:54:22,791 INFO Welcome to the CDS
2023-04-16 15:54:22,791 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-04-16 15:54:23,247 INFO Request is completed
2023-04-16 15:54:23,247 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data9/adaptor.mars.internal-1681218419.2082374-22346-10-3056153c-b2d0-4d19-b72e-73d4c8c52f93.grib to /mnt/c/MBA/data2/files_1940-01-01.grib (11.3K)
2023-04-16 15:54:26,232 INFO Download rate 3.8K/s


<xarray.Dataset>
Dimensions:    (date_ref: 24, latitude: 3, longitude: 3)
Coordinates:
  * date_ref   (date_ref) datetime64[ns] 1940-01-01 ... 1940-01-01T23:00:00
  * latitude   (latitude) float64 -10.25 -10.5 -10.75
  * longitude  (longitude) float64 -40.75 -40.5 -40.25
Data variables:
    u100       (date_ref, latitude, longitude) float32 ...
    v100       (date_ref, latitude, longitude) float32 ...
    WS100      (date_ref, latitude, longitude) float32 3.888 7.414 ... 4.907
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-04-16T15:54 GRIB to CDM+CF via cfgrib-0.9.1...


In [11]:
clima.isel(date_ref=0).to_dataframe()

date_ref      u100      v100     WS100
latitude longitude                                         
-10.25   -40.75    1940-01-01 -0.972397  3.764053  3.887628
         -40.50    1940-01-01 -5.308334  5.176163  7.414248
         -40.25    1940-01-01 -7.365952  5.911514  9.444747
-10.50   -40.75    1940-01-01 -2.349350  2.688858  3.570631
         -40.50    1940-01-01 -6.710678  4.608780  8.140887
         -40.25    1940-01-01 -7.626694  4.864639  9.046059
-10.75   -40.75    1940-01-01 -5.006577  3.012100  5.842821
         -40.50    1940-01-01 -7.981186  4.192764  9.015465
         -40.25    1940-01-01 -7.819077  4.217178  8.883837